In [ ]:
pip install lux

In [ ]:
import pandas as pd
import lux
import numpy as np
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
pip install py-AutoClean

In [ ]:
df = pd.read_csv("eeg raw.csv")

In [ ]:
df

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr
import matplotlib.pyplot as plt


# Calculate Spearman correlation coefficients for all pairs of columns
correlation_matrix = df.corr(method='spearman')

# Flatten the correlation matrix, exclude self-correlations, and sort by absolute values
correlation_values = correlation_matrix.unstack().sort_values(ascending=False)
correlation_values = correlation_values[correlation_values.index.get_level_values(0) != correlation_values.index.get_level_values(1)]

# Get the top 5 pairs
top_5_pairs = correlation_values.head(5).index

# Plot scatter plots and print monotonicity scores for the top 5 pairs
for pair in top_5_pairs:
    col1, col2 = pair

    # Calculate Spearman correlation coefficient
    spearman_coeff, _ = spearmanr(df[col1], df[col2])
    monotonicity_score = spearman_coeff ** 2

    # Print the monotonicity score
    print(f'Monotonicity Score ({col1} vs {col2}): {monotonicity_score:.2f}')


Monotonicity Score (AF4 vs AF3): 0.81
Monotonicity Score (AF3 vs AF4): 0.81
Monotonicity Score (F4 vs AF4): 0.66
Monotonicity Score (AF4 vs F4): 0.66
Monotonicity Score (O2 vs P8): 0.62


In [ ]:
#Manual Outlier Removal
import pandas as pd
from scipy import stats
summary = df.describe()
q1 = summary.loc['25%']
q3 = summary.loc['75%']
iqr = q3 - q1
# outliers
outliers = df[(df < (q1 - 1.5 * iqr)) | (df > (q3 + 1.5 * iqr))].dropna(how='all')
outlier = df.drop(outliers.index, axis=0)

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Assuming df is your DataFrame
# Create a copy of the original DataFrame
df3 = df.copy()

# Separate numerical and categorical columns
numerical_columns = df3.select_dtypes(include=['number']).columns
categorical_columns = df3.select_dtypes(include=['object']).columns

# Check if there are numerical columns
if not numerical_columns.empty:
    # Impute median for numerical columns
    numerical_imputer = SimpleImputer(strategy='median')
    df3[numerical_columns] = numerical_imputer.fit_transform(df3[numerical_columns])
if not categorical_columns.empty:
    # Impute most frequent for categorical columns
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    df3[categorical_columns] = categorical_imputer.fit_transform(df3[categorical_columns])



In [ ]:
df3

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [ ]:
outlier

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

# Assuming df is your DataFrame
# Create a copy of the original DataFrame
df4 = df.copy()

# Separate numerical and categorical columns
numerical_columns = df4.select_dtypes(include=['number']).columns
categorical_columns = df4.select_dtypes(include=['object']).columns

# Check if there are numerical columns
if not numerical_columns.empty:
    # Impute using KNN for numerical columns
    knn_imputer_numerical = KNNImputer(n_neighbors=10)
    df4[numerical_columns] = knn_imputer_numerical.fit_transform(df4[numerical_columns])

# Check if there are categorical columns
if not categorical_columns.empty:
    # One-hot encode categorical columns
    encoder = OneHotEncoder()
    df_categorical_encoded = pd.DataFrame(encoder.fit_transform(df4[categorical_columns]).toarray(), columns=encoder.get_feature_names_out(categorical_columns))

    # Concatenate the encoded categorical columns with the numerical columns
    df4 = pd.concat([df4[numerical_columns], df_categorical_encoded], axis=1)

    # Impute using KNN for all columns
    knn_imputer_all = KNNImputer(n_neighbors=10)
    df4 = pd.DataFrame(knn_imputer_all.fit_transform(df4), columns=df4.columns)



In [ ]:
df4

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr
import matplotlib.pyplot as plt


# Calculate Spearman correlation coefficients for all pairs of columns
correlation_matrix = df.corr(method='spearman')

# Flatten the correlation matrix, exclude self-correlations, and sort by absolute values
correlation_values = correlation_matrix.unstack().sort_values(ascending=False)
correlation_values = correlation_values[correlation_values.index.get_level_values(0) != correlation_values.index.get_level_values(1)]

# Get the top 5 pairs
top_5_pairs = correlation_values.head(5).index

# Plot scatter plots and print monotonicity scores for the top 5 pairs
for pair in top_5_pairs:
    col1, col2 = pair

    # Calculate Spearman correlation coefficient
    spearman_coeff, _ = spearmanr(df[col1], df[col2])
    monotonicity_score = spearman_coeff ** 2

    # Print the monotonicity score
    print(f'Monotonicity Score ({col1} vs {col2}): {monotonicity_score:.2f}')


Monotonicity Score (Education-num vs Hours-per-week): 0.03
Monotonicity Score (Hours-per-week vs Education-num): 0.03
Monotonicity Score (Hours-per-week vs Age): 0.02
Monotonicity Score (Age vs Hours-per-week): 0.02
Monotonicity Score (Age vs Capital-gain): 0.02


In [ ]:
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

# List of DataFrames
dataframes = [df, df3,df4, outlier]

# Iterate through all DataFrames
for idx, current_df in enumerate(dataframes, start=1):
    # Calculate Spearman correlation coefficients for all pairs of columns
    # Calculate Spearman correlation coefficients for all pairs of columns
    correlation_matrix = current_df.corr(method='spearman')

    # Flatten the correlation matrix, exclude self-correlations, and sort by absolute values
    correlation_values = correlation_matrix.unstack().sort_values(ascending=False)
    correlation_values = correlation_values[correlation_values.index.get_level_values(0) != correlation_values.index.get_level_values(1)]

    # Get the top 5 pairs
    top_5_pairs = correlation_values.head(4).index

    # Plot scatter plots and print monotonicity scores for the top 5 pairs
    for pair in top_5_pairs:
        col1, col2 = pair

        # Calculate Spearman correlation coefficient
        spearman_coeff, _ = spearmanr(current_df[col1], current_df[col2])
        monotonicity_score = spearman_coeff ** 2

        # Print the monotonicity score
        print(f'Monotonicity Score (DataFrame {idx} - {col1} vs {col2}): {monotonicity_score:.2f}')


Monotonicity Score (DataFrame 1 - Education-num vs Hours-per-week): 0.03
Monotonicity Score (DataFrame 1 - Hours-per-week vs Education-num): 0.03
Monotonicity Score (DataFrame 1 - Hours-per-week vs Age): 0.02
Monotonicity Score (DataFrame 1 - Age vs Hours-per-week): 0.02
Monotonicity Score (DataFrame 2 - Education-num vs Hours-per-week): 0.03
Monotonicity Score (DataFrame 2 - Hours-per-week vs Education-num): 0.03
Monotonicity Score (DataFrame 2 - Hours-per-week vs Age): 0.02
Monotonicity Score (DataFrame 2 - Age vs Hours-per-week): 0.02
Monotonicity Score (DataFrame 3 - Occupation_nan vs Workclass_nan): 1.00
Monotonicity Score (DataFrame 3 - Workclass_nan vs Occupation_nan): 1.00
Monotonicity Score (DataFrame 3 - Marital-status_ married-civ-spouse vs Relationship_ husband): 0.80
Monotonicity Score (DataFrame 3 - Relationship_ husband vs Marital-status_ married-civ-spouse): 0.80
Monotonicity Score (DataFrame 4 - Education-num vs Hours-per-week): 0.01
Monotonicity Score (DataFrame 4 - H

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import numpy as np

# List of DataFrames
dataframes = [df, df3, df4, outlier]

# Function to analyze DataFrame
def analyze_dataframe(df):
    # Calculate Spearman correlation coefficients for all pairs of columns
    correlation_matrix = df.corr(method='spearman')

    # Flatten the correlation matrix, exclude self-correlations, and sort by absolute values
    correlation_values = correlation_matrix.unstack().sort_values(ascending=False)
    correlation_values = correlation_values[correlation_values.index.get_level_values(0) != correlation_values.index.get_level_values(1)]

    # Get the top 5 pairs
    top_5_pairs = correlation_values.head(4).index

    # Plot scatter plots and print monotonicity scores for the top 5 pairs
    for pair in top_5_pairs:
        col1, col2 = pair

        # Calculate Spearman correlation coefficient
        spearman_coeff, _ = spearmanr(df[col1], df[col2])
        monotonicity_score = spearman_coeff ** 2

        # Print the monotonicity score
        print(f'Monotonicity Score (DataFrame - {col1} vs {col2}): {monotonicity_score:.2f}')

    # Iterate through all columns for unevenness analysis
    for column_name in df.columns:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(df[column_name]):
            # Calculate the Euclidean distance (L2 norm) between V and V_flat for bar chart
            V = df[column_name].values
            V_flat = np.full_like(V, np.mean(V))  # Flat uniform distribution

            euclidean_distance = np.linalg.norm(V - V_flat)

            # Print the unevenness score
            print(f'Unevenness Score (DataFrame - {column_name}): {euclidean_distance:.2f}')

# Iterate through all DataFrames
for idx, current_df in enumerate(dataframes, start=1):
    print(f"\nAnalysis for DataFrame {idx}:")
    analyze_dataframe(current_df)



Analysis for DataFrame 1:
Monotonicity Score (DataFrame - Education-num vs Hours-per-week): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Education-num): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Age): 0.02
Monotonicity Score (DataFrame - Age vs Hours-per-week): 0.02
Unevenness Score (DataFrame - Age): 2463.57
Unevenness Score (DataFrame - Fnlwgt): 19045849.31
Unevenness Score (DataFrame - Education-num): 464.46
Unevenness Score (DataFrame - Capital-gain): 1332630.89
Unevenness Score (DataFrame - Capital-loss): 72711.73
Unevenness Score (DataFrame - Hours-per-week): 2229.42

Analysis for DataFrame 2:
Monotonicity Score (DataFrame - Education-num vs Hours-per-week): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Education-num): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Age): 0.02
Monotonicity Score (DataFrame - Age vs Hours-per-week): 0.02
Unevenness Score (DataFrame - Age): 2461.33
Unevenness Score (DataFrame - Fnlwgt): 19045849.31
Unevenness

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import numpy as np

# List of DataFrames
dataframes = [df, df3, df4, outlier]

# Function to analyze DataFrame
def analyze_dataframe(df):
    scores = []

    # Calculate Spearman correlation coefficients for all pairs of columns
    correlation_matrix = df.corr(method='spearman')

    # Flatten the correlation matrix, exclude self-correlations, and sort by absolute values
    correlation_values = correlation_matrix.unstack().sort_values(ascending=False)
    correlation_values = correlation_values[correlation_values.index.get_level_values(0) != correlation_values.index.get_level_values(1)]

    # Get the top 5 pairs
    top_5_pairs = correlation_values.head(4).index

    # Print the top 5 monotonicity scores
    for pair in top_5_pairs:
        col1, col2 = pair

        # Calculate Spearman correlation coefficient
        spearman_coeff, _ = spearmanr(df[col1], df[col2])
        monotonicity_score = spearman_coeff ** 2

        scores.append((f'Monotonicity Score (DataFrame - {col1} vs {col2})', monotonicity_score))

    # Sort scores by the monotonicity score in descending order
    scores.sort(key=lambda x: x[1], reverse=True)

    print("\nTop 4 Monotonicity Scores:")
    for i in range(min(4, len(scores))):
        print(f'{scores[i][0]}: {scores[i][1]:.2f}')

    # Iterate through all columns for unevenness analysis
    for column_name in df.columns:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(df[column_name]):
            # Calculate the Euclidean distance (L2 norm) between V and V_flat for bar chart
            V = df[column_name].values
            V_flat = np.full_like(V, np.mean(V))  # Flat uniform distribution

            euclidean_distance = np.linalg.norm(V - V_flat)

            scores.append((f'Unevenness Score (DataFrame - {column_name})', euclidean_distance))

    # Sort scores by the unevenness score in descending order
    scores.sort(key=lambda x: x[1], reverse=True)

    print("\nTop 4 Unevenness Scores:")
    for i in range(min(4, len(scores))):
        print(f'{scores[i][0]}: {scores[i][1]:.2f}')

# Iterate through all DataFrames
for idx, current_df in enumerate(dataframes, start=1):
    print(f"\nAnalysis for DataFrame {idx}:")
    analyze_dataframe(current_df)



Analysis for DataFrame 1:

Top 4 Monotonicity Scores:
Monotonicity Score (DataFrame - AF4 vs AF3): 0.81
Monotonicity Score (DataFrame - AF3 vs AF4): 0.81
Monotonicity Score (DataFrame - F4 vs AF4): 0.66
Monotonicity Score (DataFrame - AF4 vs F4): 0.66

Top 4 Unevenness Scores:
Unevenness Score (DataFrame - AF4): 721026.86
Unevenness Score (DataFrame - FC5): 638429.11
Unevenness Score (DataFrame - O1): 563101.53
Unevenness Score (DataFrame - P7): 357961.26

Analysis for DataFrame 2:

Top 4 Monotonicity Scores:
Monotonicity Score (DataFrame - AF4 vs AF3): 0.81
Monotonicity Score (DataFrame - AF3 vs AF4): 0.81
Monotonicity Score (DataFrame - F4 vs AF4): 0.66
Monotonicity Score (DataFrame - AF4 vs F4): 0.66

Top 4 Unevenness Scores:
Unevenness Score (DataFrame - AF4): 721026.86
Unevenness Score (DataFrame - FC5): 638429.11
Unevenness Score (DataFrame - O1): 563101.53
Unevenness Score (DataFrame - P7): 357961.26

Analysis for DataFrame 3:

Top 4 Monotonicity Scores:
Monotonicity Score (Dat

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import numpy as np

# List of DataFrames
dataframes = [df, df3, df4, outlier]

# Function to analyze DataFrame
def analyze_dataframe(df):
    scores = []

    # Calculate Spearman correlation coefficients for all pairs of columns
    correlation_matrix = df.corr(method='spearman')

    # Flatten the correlation matrix, exclude self-correlations, and sort by absolute values
    correlation_values = correlation_matrix.unstack().sort_values(ascending=False)
    correlation_values = correlation_values[correlation_values.index.get_level_values(0) != correlation_values.index.get_level_values(1)]

    # Get the top 5 pairs for monotonicity analysis
    top_5_pairs_monotonicity = correlation_values.head(5).index

    # Print the top 5 monotonicity scores
    for pair in top_5_pairs_monotonicity:
        col1, col2 = pair

        # Calculate Spearman correlation coefficient
        spearman_coeff, _ = spearmanr(df[col1], df[col2])
        monotonicity_score = spearman_coeff ** 2

        scores.append((f'Monotonicity Score (DataFrame - {col1} vs {col2})', monotonicity_score))

    # Sort scores by the monotonicity score in descending order
    scores.sort(key=lambda x: x[1], reverse=True)

    print("\nTop 5 Monotonicity Scores:")
    for i in range(min(5, len(scores))):
        print(f'{scores[i][0]}: {scores[i][1]:.2f}')

    # Get all pairs of columns for unevenness analysis
    all_column_pairs_unevenness = [(col1, col2) for col1 in df.columns for col2 in df.columns if pd.api.types.is_numeric_dtype(df[col1]) and pd.api.types.is_numeric_dtype(df[col2])]

    # Calculate unevenness scores for all pairs of columns
    for pair in all_column_pairs_unevenness:
        col1, col2 = pair

        # Calculate the Euclidean distance (L2 norm) between V and V_flat for bar chart
        V = df[col1].values
        V_flat = np.full_like(V, np.mean(V))  # Flat uniform distribution

        euclidean_distance = np.linalg.norm(V - V_flat)

        scores.append((f'Unevenness Score (DataFrame - {col1} vs {col2})', euclidean_distance))

    # Sort scores by the unevenness score in descending order
    scores.sort(key=lambda x: x[1], reverse=True)

    print("\nTop 5 Unevenness Scores:")
    for i in range(min(5, len(scores))):
        print(f'{scores[i][0]}: {scores[i][1]:.2f}')

# Iterate through all DataFrames
for idx, current_df in enumerate(dataframes, start=1):
    print(f"\nAnalysis for DataFrame {idx}:")
    analyze_dataframe(current_df)



Analysis for DataFrame 1:

Top 5 Monotonicity Scores:
Monotonicity Score (DataFrame - Education-num vs Hours-per-week): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Education-num): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Age): 0.02
Monotonicity Score (DataFrame - Age vs Hours-per-week): 0.02
Monotonicity Score (DataFrame - Age vs Capital-gain): 0.02

Top 5 Unevenness Scores:
Unevenness Score (DataFrame - Fnlwgt vs Age): 19045849.31
Unevenness Score (DataFrame - Fnlwgt vs Fnlwgt): 19045849.31
Unevenness Score (DataFrame - Fnlwgt vs Education-num): 19045849.31
Unevenness Score (DataFrame - Fnlwgt vs Capital-gain): 19045849.31
Unevenness Score (DataFrame - Fnlwgt vs Capital-loss): 19045849.31

Analysis for DataFrame 2:

Top 5 Monotonicity Scores:
Monotonicity Score (DataFrame - Education-num vs Hours-per-week): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Education-num): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Age): 0.02
Monotonicity Scor

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr, skew
import matplotlib.pyplot as plt
import numpy as np

# List of DataFrames
dataframes = [df, df3, df4, outlier]

# Function to analyze DataFrame
def analyze_dataframe(df):
    # Calculate Spearman correlation coefficients for all pairs of columns
    correlation_matrix = df.corr(method='spearman')

    # Flatten the correlation matrix, exclude self-correlations, and sort by absolute values
    correlation_values = correlation_matrix.unstack().sort_values(ascending=False)
    correlation_values = correlation_values[correlation_values.index.get_level_values(0) != correlation_values.index.get_level_values(1)]

    # Get the top 5 pairs for monotonicity analysis
    top_5_pairs_monotonicity = correlation_values.head(5).index

    # Plot scatter plots and print monotonicity scores for the top 5 pairs
    for pair in top_5_pairs_monotonicity:
        col1, col2 = pair

        # Calculate Spearman correlation coefficient
        spearman_coeff, _ = spearmanr(df[col1], df[col2])
        monotonicity_score = spearman_coeff ** 2

        # Print the monotonicity score
        print(f'Monotonicity Score (DataFrame - {col1} vs {col2}): {monotonicity_score:.2f}')

    # Get all pairs of columns for unevenness analysis
    all_column_pairs_unevenness = [(col1, col2) for col1 in df.columns for col2 in df.columns if pd.api.types.is_numeric_dtype(df[col1]) and pd.api.types.is_numeric_dtype(df[col2])]

    # Calculate unevenness scores for all pairs of columns
    unevenness_scores = []

    for pair in all_column_pairs_unevenness:
        col1, col2 = pair

        # Calculate the Euclidean distance (L2 norm) between V and V_flat for bar chart
        V = df[col1].values
        V_flat = np.full_like(V, np.mean(V))  # Flat uniform distribution

        euclidean_distance = np.linalg.norm(V - V_flat)

        # Append the unevenness score and pair to the list
        unevenness_scores.append((pair, euclidean_distance))

    # Get the top 5 pairs for unevenness analysis
    top_5_pairs_unevenness = sorted(unevenness_scores, key=lambda x: x[1])[:5]

    # Iterate through the top 5 pairs for unevenness analysis
    for pair, unevenness_score in top_5_pairs_unevenness:
        col1, col2 = pair

        # Print the unevenness score
        print(f'Unevenness Score (DataFrame - {col1} vs {col2}): {unevenness_score:.2f}')

    # Get all pairs of columns for skewness analysis
    all_column_pairs_skewness = [(col1, col2) for col1 in df.columns for col2 in df.columns if pd.api.types.is_numeric_dtype(df[col1]) and pd.api.types.is_numeric_dtype(df[col2])]

    # Calculate skewness for all pairs of columns
    skewness_scores = []

    for pair in all_column_pairs_skewness:
        col1, col2 = pair

        # Calculate skewness using scipy.stats.skew
        skewness_score_col1 = skew(df[col1])
        skewness_score_col2 = skew(df[col2])

        # Append the skewness scores and pair to the list
        skewness_scores.append((pair, skewness_score_col1, skewness_score_col2))

    # Get the top 5 pairs for skewness analysis
    top_5_pairs_skewness = sorted(skewness_scores, key=lambda x: abs(x[1]) + abs(x[2]), reverse=True)[:5]

    # Iterate through the top 5 pairs for skewness analysis
    for pair, skewness_score_col1, skewness_score_col2 in top_5_pairs_skewness:
        col1, col2 = pair

        # Print the skewness scores
        print(f'Skewness Score (DataFrame - {col1} vs {col2}): {skewness_score_col1:.2f} and {skewness_score_col2:.2f}')

# Iterate through all DataFrames
for idx, current_df in enumerate(dataframes, start=1):
    print(f"\nAnalysis for DataFrame {idx}:")
    analyze_dataframe(current_df)



Analysis for DataFrame 1:
Monotonicity Score (DataFrame - Education-num vs Hours-per-week): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Education-num): 0.03
Monotonicity Score (DataFrame - Hours-per-week vs Age): 0.02
Monotonicity Score (DataFrame - Age vs Hours-per-week): 0.02
Monotonicity Score (DataFrame - Age vs Capital-gain): 0.02
Unevenness Score (DataFrame - Education-num vs Age): 464.46
Unevenness Score (DataFrame - Education-num vs Fnlwgt): 464.46
Unevenness Score (DataFrame - Education-num vs Education-num): 464.46
Unevenness Score (DataFrame - Education-num vs Capital-gain): 464.46
Unevenness Score (DataFrame - Education-num vs Capital-loss): 464.46
Skewness Score (DataFrame - Capital-gain vs Capital-gain): 11.95 and 11.95
Skewness Score (DataFrame - Capital-gain vs Capital-loss): 11.95 and 4.59
Skewness Score (DataFrame - Capital-loss vs Capital-gain): 4.59 and 11.95
Skewness Score (DataFrame - Fnlwgt vs Capital-gain): 1.45 and 11.95
Skewness Score (DataFrame - 